<a href="https://colab.research.google.com/github/siroba/Movie-recommendation-system/blob/main/MovieRecommendationSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Movie recomemender system

##Import libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA 
from sklearn.manifold import TSNE
import numpy as np

%tensorflow_version 1.x

from keras import optimizers
import tensorflow as tf


from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import  Dense
from keras.layers import Dropout
from keras.utils.np_utils import to_categorical

TensorFlow 1.x selected.


Using TensorFlow backend.


##Download the dataset

In [2]:
!gdown --id 19hgYPmZAJQnHMTqXpCdihQeJKVsuImoS
!unzip 'ml-100k.zip' -d '.'

Downloading...
From: https://drive.google.com/uc?id=19hgYPmZAJQnHMTqXpCdihQeJKVsuImoS
To: /content/ml-100k.zip
100% 4.92M/4.92M [00:00<00:00, 39.1MB/s]
Archive:  ml-100k.zip
   creating: ./ml-100k/
  inflating: ./ml-100k/allbut.pl     
  inflating: ./ml-100k/mku.sh        
  inflating: ./ml-100k/README        
  inflating: ./ml-100k/u.data        
  inflating: ./ml-100k/u.genre       
  inflating: ./ml-100k/u.info        
  inflating: ./ml-100k/u.item        
  inflating: ./ml-100k/u.occupation  
  inflating: ./ml-100k/u.user        
  inflating: ./ml-100k/u1.base       
  inflating: ./ml-100k/u1.test       
  inflating: ./ml-100k/u2.base       
  inflating: ./ml-100k/u2.test       
  inflating: ./ml-100k/u3.base       
  inflating: ./ml-100k/u3.test       
  inflating: ./ml-100k/u4.base       
  inflating: ./ml-100k/u4.test       
  inflating: ./ml-100k/u5.base       
  inflating: ./ml-100k/u5.test       
  inflating: ./ml-100k/ua.base       
  inflating: ./ml-100k/ua.test       
  in

##Functions

In [4]:
def plot_history(history, validation=False):
 plt.figure()
 plt.xlabel('Epoch')
 plt.ylabel('loss_function_error')
 plt.plot(history.epoch, np.array(history.history['loss']), label='Train Loss')
 if validation:
  plt.plot(history.epoch, np.array(history.history['val_loss']), label='Validation Loss')
 plt.legend()
 limite_superior = (max(history.history['loss'])-min(history.history['loss']))/2
 plt.ylim([0, max(limite_superior, 2)])

In [5]:
#Function to do the correlation
def relation(data):
  plt.scatter(data.Age, data.rating, alpha=0.25)
  plt.show()
  plt.figure(figsize=(60,30))
  cc=np.corrcoef(data.Age, data.rating, rowvar=True)[0, 1]
  return cc

#Function used to load the data
def load_data(data):
  users = pd.read_csv("/content/ml-100k/u.user", sep='|', names = ["User ID", "Age","Gender","Profession","Zip"]) # Load data
  users = pd.concat([users, pd.get_dummies(users.Profession, prefix='Profession')], axis = 1)
  users = pd.concat([users, pd.get_dummies(users.Gender, drop_first=True)], axis = 1)
  users = users.drop(['Profession','Gender'], axis = 1)

  movies = pd.read_csv('/content/ml-100k/u.item', sep='|', encoding="latin1", names=["Movie ID", "Title", "Release date", "Video Release Date", "IMDb URL", "unknown", "Action", "Adventure", "Animation", "Children", "Comedy", "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror", "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"]).drop(["Video Release Date", "unknown", "IMDb URL", "Title", "Release date"], axis=1)

  data = data.merge(users, on="User ID")
  data = data.merge(movies, on="Movie ID")
  data.drop(["User ID", "Zip","Movie ID", "Timestamp"], axis=1, inplace=True)

  return data
#Function used to load  all the data the data
def load_AllData(data):
  users = pd.read_csv("/content/ml-100k/u.user", sep='|', names = ["User ID", "Age","Gender","Profession","Zip"]) # Load data
  users = users.drop(['Profession','Gender'], axis = 1)

  movies = pd.read_csv('/content/ml-100k/u.item', sep='|', encoding="latin1", names=["Movie ID", "Title", "Release date", "Video Release Date", "IMDb URL", "unknown", "Action", "Adventure", "Animation", "Children", "Comedy", "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror", "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"]).drop(["Video Release Date", "unknown", "IMDb URL", "Release date"], axis=1)

  data = data.merge(users, on="User ID")
  data = data.merge(movies, on="Movie ID")
  data.drop([ "Zip", "Timestamp"], axis=1, inplace=True)


  return data

##Collect the data

In [6]:
data =pd.read_csv('/content/ml-100k/u1.base', sep='\t', names = ["User ID", "Movie ID","Rating","Timestamp"])
dataTest =pd.read_csv('/content/ml-100k/u1.test', sep='\t', names = ["User ID", "Movie ID","Rating","Timestamp"])
data= load_data(data)
dataTest=load_data(dataTest)

#data.head(4)
dataTest.head(20)

,Rating,Age,Profession_administrator,Profession_artist,Profession_doctor,Profession_educator,Profession_engineer,Profession_entertainment,Profession_executive,Profession_healthcare,Profession_homemaker,Profession_lawyer,Profession_librarian,Profession_marketing,Profession_none,Profession_other,Profession_programmer,Profession_retired,Profession_salesman,Profession_scientist,Profession_student,Profession_technician,Profession_writer,M,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,5,24,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,5,29,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,3,31,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,4,39,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,4,60,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
5,4,48,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
6,3,24,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
7,4,57,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
8,3,23,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
9,4,49,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0


In [7]:
data.corr().iloc[:,:1]

,Rating
Rating,1.000000
Age,0.049143
Profession_administrator,0.023330
Profession_artist,0.006175
Profession_doctor,0.011188
Profession_educator,0.043210
Profession_engineer,0.000479
Profession_entertainment,-0.010154
Profession_executive,-0.016524
Profession_healthcare,-0.099006


##Deep learning
The AI will learn (using deep learning) a classifier from users and movies to ratings (4 and 5 stars are going to be considered “like”, while the other scores are “no-like”). 

## Prepare the data for training

In [8]:
trainX = data.iloc[:, 1:] #All the rows and all the columns except the 1º
trainY=data.iloc[:,:1] #All the rows and only the 1º column

trainY.head(2)

testX = dataTest.iloc[:, 1:] #All the rows and all the columns except the 1º
testY = dataTest.iloc[:,:1] #All the rows and only the 1º column

testY.head(2) #test if the selection od data is correct

,Rating
0,5
1,5


Split the rating into like ( 4 or 5) or no like (3,2,1)

In [9]:
testY.loc[testY['Rating'] <4, 'Rating'] = 0
testY.loc[testY['Rating'] >=4, 'Rating'] = 1

trainY.loc[trainY['Rating'] <4, 'Rating'] = 0
trainY.loc[trainY['Rating'] >=4, 'Rating'] = 1

print(trainY)

       Rating
0           1
1           1
2           1
3           0
4           1
...       ...
79995       0
79996       0
79997       0
79998       0
79999       0

[80000 rows x 1 columns]


Create the CNN

In [10]:
batch_size = 80
lr= 0.0001
epochs= 60

In [13]:
# We create a sequential model, composed of a linear sequence of layers
model = Sequential()
n_classes=1

model.add(Dense(1000, activation='relu'))

model.add(Dense(500, activation='relu'))
model.add(Dense(10, activation='relu'))


model.add(Dense(n_classes, activation='elu'))



model.compile(
  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
  optimizer=optimizers.Nadam(lr)
)



In [14]:
history = model.fit(trainX.values, trainY.values, batch_size,epochs,validation_data=(testX.values,testY.values))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 80000 samples, validate on 20000 samples
Epoch 1/60
80000/80000 [==============================] - 13s 164us/step - loss: 0.6922 - val_loss: 0.6901
Epoch 2/60
80000/80000 [==============================] - 15s 182us/step - loss: 0.6898 - val_loss: 0.6881
Epoch 3/60
80000/80000 [==============================] - 16s 195us/step - loss: 0.6887 - val_loss: 0.6870
Epoch 4/60
80000/80000 [==============================] - 22s 275us/step - loss: 0.6882 - val_loss: 0.6864
Epoch 5/60
80000/80000 [==============================] - 13s 160us/step - loss: 0.6880 - val_loss: 0.6861
Epoch 6/60
80000/80000 [==============================] - 13s 166us/step - loss: 0.6879 - val_loss: 0.6860
Epoch 7/60
80000/80000 [==============================] - 13s 158us/step - loss: 0.6878 - val_loss: 0.6859
Epoch 8/60
80000/80000 [==============================] - 13s 158us/step - loss: 0.6878 - val_loss: 0.6858

KeyboardInterrupt: ignored

In [ ]:
# train
traza_train = np.array(history.history['loss'])
print("-- Minimunm error TRAIN: {:.6f}".format(min(traza_train)))
print("      Avg 10 últimas epochs: ** {:.6f} **".format(np.mean(traza_train[-10:])))
print("      Desv. 10 últimas epochs: {:.6f}".format(np.std(traza_train[-10:])))
# validation
traza_test = np.array(history.history['val_loss'])
print("-- Minimun error TEST: {:.6f}".format(min(traza_test)))
print("      Avg 10 últimas epochs: ** {:.6f} **".format(np.mean(traza_test[-10:])))
print("      Desv. 10 últimas epochs: {:.6f}".format(np.std(traza_test[-10:])))

clase_pred = model.predict(testX)
pred = pd.DataFrame(np.hstack([testY, clase_pred]), columns=['Y_real', 'Y_predicted'])
pred.to_excel('score_pred.xlsx', sheet_name="real vs. predicted")

In [ ]:
plot_history(history, True)

##Regression model


We will use a regression model instead of a CNN for the prediction. The AI is going to use the number of stars as a real number to be predicted. 

In [ ]:
tf.random.set_random_seed(2032) # Set keras/tensorflow seed
n_input = trainX.shape[1] # Cells in the input layer (input variables)
n_hidden1 =600
n_hidden2 = 600
n_hidden3= 600
n_output = 1 # Cells in the output layer (a numeric value)
model = Sequential()
model.add(Dense(n_hidden1, input_dim=n_input,activation='relu', name="hidden1_layer"))
model.add(Dropout(0.2))
model.add(Dense(n_hidden2,activation='relu', name="hidden2_layer"))
model.add(Dense(n_hidden3, activation='relu', name="hidden3_layer"))
model.add(Dense(n_output, name="output_layer"))
learning_rate = 1e-4

opt = tf.keras.optimizers.SGD(lr=learning_rate)
model.compile(loss='mean_squared_error', optimizer=optimizers.Nadam(lr))

Visualization of the model

In [ ]:
model.summary()

batch = 180 #  batch size 
n_epochs = 28 # Nº epochs 900

history = model.fit(trainX, trainY, batch_size=batch, epochs=n_epochs, verbose=2,
 validation_data=(testX, testY))

# train
traza_train = np.array(history.history['loss'])
print("-- Mínimo error TRAIN: {:.6f}".format(min(traza_train)))
print(" Media 10 últimas epochs: ** {:.6f} **".format(np.mean(traza_train[-10:])))
print(" Desv. 10 últimas epochs: {:.6f}".format(np.std(traza_train[-10:])))

#validation
traza_test = np.array(history.history['val_loss'])
print("-- Mínimo error TEST: {:.6f}".format(min(traza_test)))
print(" Media 10 últimas epochs: ** {:.6f} **".format(np.mean(traza_test[-10:])))
print(" Desv. 10 últimas epochs: {:.6f}".format(np.std(traza_test[-10:])))

 #graph
plot_history(history, validation=True)
plt.show()
#prediction proposed by the model for the validation examples
y_predicted = model.predict(testX)

# the real value and the prediction are stored in an excel (in case you want to see them)
pred = pd.DataFrame(np.hstack([testY, y_predicted]), columns=['Y_true', 'y_predicted'])
pred.to_excel('results.xlsx', sheet_name="real vs. predicted")

# EAM is Mean Absolute Error: abs(Y_true - y_predicted)
# A very simple system that is not at all intelligent is to always predict the mean value of
# Quality observed in the training set. Doing that you would have an EAM:
EAM_sys_mean = sum(abs(pred.Y_true - testY.mean().values[0]))/pred.shape[0]
print("Average absolute error using the AVERAGE system: %.4f (always predicts %.2f)"
 % (EAM_sys_mean, testY.mean().values[0]))

#if it is predicted using the model learned by the network, the EAM obtained will be:
EAM_sys_red = sum(abs(pred.Y_true - pred.y_predicted))/pred.shape[0]
print("Mean absolute error using the trained neural network: %.4f" % EAM_sys_red)